In [1]:
import scrapy
import scrapy.crawler as crawler
from multiprocessing import Process, Queue
from twisted.internet import reactor
import json
import time
import unidecode

import pandas as pd

from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.support.ui import WebDriverWait

from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys

In [2]:
rome_codes = pd.read_csv('data/CODES ROME - Sheet1.csv')
rome_codes

,Intitulé du poste,Code ROME
0,Adjointe RH,M1501
1,Animateur sécurité,K1205
2,Animateur qualité,H1502
3,Apprenti laboratoire,H1503
4,Apprenti sécurité,K2503
5,Conducteur d'installation fruits,H2102
6,Conducteur d'installation process,H2906
7,Conducteur de ligne,L1304
8,Conducteur de palettiseur centralisé,H3301
9,Coordinateur technique,H1101


In [3]:
base_url = 'https://candidat.pole-emploi.fr/marche-du-travail/fichemetierrome?codeRome='

In [4]:
options = Options()
driver = webdriver.Firefox(options=options, executable_path=r'C:/driver/geckodriver.exe')

In [5]:
driver.get(base_url + 'H1506')

In [6]:
mobility = WebDriverWait(driver, 10).until(EC.presence_of_element_located(
    (By.XPATH, "/html/body/div[4]/div/div/div[2]/form/div[3]/div[3]/div/div/div/div[2]/ul/li[3]/a")))
mobility.click()

In [7]:
strong_match = mobility.find_element_by_xpath("/html/body/div[4]/div/div/div[2]/form/div[3]/div[3]/div/div/div/div[2]/div/div[3]/div[2]/div/table/tbody")

In [8]:
for tr in strong_match.find_elements_by_tag_name('tr'):
    print(str(tr.find_elements_by_tag_name('td')[1].text).split('-')[1])


 Pilotage d'unité élémentaire de production mécanique ou de travail des métaux
 Intervention technique en laboratoire d'analyse industrielle
 Intervention technique en méthodes et industrialisation
 Inspection de conformité
 Intervention technique en Hygiène Sécurité Environnement 


In [9]:
need_trainning = mobility.find_element_by_xpath("/html/body/div[4]/div/div/div[2]/form/div[3]/div[3]/div/div/div/div[2]/div/div[3]/div[4]/div/table/tbody")

In [10]:
for tr in need_trainning.find_elements_by_tag_name('tr'):
    print(str(tr.find_elements_by_tag_name('td')[1].text).split('-')[1])

 Rédaction technique
 Intervention technique en études, recherche et développement
 Formation professionnelle
 Direction de laboratoire d'analyse industrielle
 Management et ingénierie qualité industrielle
 Intervention technique en contrôle essai qualité en électricité et électronique
 Assistance et support technique client


In [11]:
results = []

for code in rome_codes['Code ROME']:
    
    driver.get(base_url + code)
    
    mobility = WebDriverWait(driver, 10).until(EC.presence_of_element_located(
    (By.XPATH, "/html/body/div[4]/div/div/div[2]/form/div[3]/div[3]/div/div/div/div[2]/ul/li[3]/a")))
    mobility.click()
    
    strong_match = mobility.find_element_by_xpath("/html/body/div[4]/div/div/div[2]/form/div[3]/div[3]/div/div/div/div[2]/div/div[3]/div[2]/div/table/tbody")
    for tr in strong_match.find_elements_by_tag_name('tr'):
            results.append(
                {
                    'jobs': rome_codes[rome_codes['Code ROME'].str.startswith(code)]['Intitulé du poste '].values[0],
                    'rome_code': code,
                    'jobs_suggested': str(tr.find_elements_by_tag_name('td')[1].text).split('-')[1],
                    'rome_code_suggested': str(tr.find_elements_by_tag_name('td')[1].text).split('-')[0],
                    'type': 'Strong'
                }
            )
        #print(str(tr.find_elements_by_tag_name('td')[1].text).split('-')[1])
    
    need_trainning = mobility.find_element_by_xpath("/html/body/div[4]/div/div/div[2]/form/div[3]/div[3]/div/div/div/div[2]/div/div[3]/div[4]/div/table/tbody")
    for tr in need_trainning.find_elements_by_tag_name('tr'):
            results.append(
                {
                    'jobs': rome_codes[rome_codes['Code ROME'].str.startswith(code)]['Intitulé du poste '].values[0],
                    'rome_code': code,
                    'jobs_suggested': str(tr.find_elements_by_tag_name('td')[1].text).split('-')[1],
                    'rome_code_suggested': str(tr.find_elements_by_tag_name('td')[1].text).split('-')[0],
                    'type': 'Week'
                }
            )
        
        #print(str(tr.find_elements_by_tag_name('td')[1].text).split('-')[1])
         
df = pd.DataFrame(results)
df

,jobs,jobs_suggested,rome_code,rome_code_suggested,type
0,Adjointe RH,Assistanat de direction,M1501,M1604,Strong
1,Adjointe RH,Conseil en emploi et insertion socioprofessio...,M1501,K1801,Week
2,Adjointe RH,Coordination pédagogique,M1501,K2102,Week
3,Adjointe RH,Formation professionnelle,M1501,K2111,Week
4,Adjointe RH,Formation professionnelle,M1501,K2111,Week
5,Adjointe RH,Comptabilité,M1501,M1203,Week
6,Adjointe RH,Développement des ressources humaines,M1501,M1502,Week
7,Adjointe RH,Management des ressources humaines,M1501,M1503,Week
8,Animateur sécurité,Médiation sociale et facilitation de la vie e...,K1205,K1204,Strong
9,Animateur sécurité,Accueil et renseignements,K1205,M1601,Strong


In [12]:
pd.DataFrame(results).to_csv('data/rome_suggested.csv', index=False)

In [13]:
pd.read_csv('data/rome_suggested.csv').to_clipboard()